In [1]:
import pandas as pd
import numpy as np
from datetime import datetime 
from scipy.sparse import hstack,csr_matrix

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

In [2]:
DTrain = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Applied Ai/PROJECT/New_Train_Data.pkl')
DTest = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Applied Ai/PROJECT/New_Test_Data.pkl')

In [3]:
DTrain.head(2)

,ncodpers,renta,a_ind_empleado,b_ind_empleado,f_ind_empleado,n_ind_empleado,s_ind_empleado,ad_pais_residencia,ae_pais_residencia,al_pais_residencia,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,15889,0.555447,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,15890,0.404941,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,1,1,1


In [4]:
DTest.head(2)

,ncodpers,renta,a_ind_empleado,b_ind_empleado,f_ind_empleado,n_ind_empleado,s_ind_empleado,ad_pais_residencia,ae_pais_residencia,al_pais_residencia,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,15889,0.555447,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
1,15890,0.404941,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,1,1


In [5]:
#id and Customer Data
XTrain_Cust = DTrain.iloc[:,:-24]
XTest_Cust = DTest.iloc[:,:-24]

#id and Product Data
ilocPD = [0]+list(np.arange(-24,0))
YTrain_Prod = DTrain.iloc[:,ilocPD]
YTest_Prod = DTest.iloc[:,ilocPD]

print(XTrain_Cust.shape)
print(XTrain_Cust.shape)
print(YTrain_Prod.shape)
print(YTest_Prod.shape)

(944901, 391)
(944901, 391)
(944901, 25)
(931453, 25)


**Modelling:**
- Here, We are going to use advantages of both Content based filtering and Collaborative filtering
- In First Model, We'll use **Collborative filtering**
 - In which, Model will be trained only on Product Data.
 - It'll use product purchasing behaviour of Customers into account
- In Second Model,We'll be training **Content based filtering** Model
 - In which, Model will be trained on Customer Related Data
 - This Model will user Customer Information to Predict Probabilities

- In Our Final Customized Model, We'll be taking Average of the probabilities of both the Models to predct the output.
- We'll be using Logistic Regression to train both models,
- Why Logistic Regression?
 - Binary Classification in each iteration
 - Gives Probability Score
 - Workd well on High Dimentional Data
 - Multicollinearity does not affect

**Model 1**

In [6]:
#Store List of Models
models = {}
#Stores List of Probabilities of each model
model_preds = {}
#creating default dictionary to store ids of predictions
id_preds = defaultdict(list)
#Stores unique ids

ids = YTrain_Prod['ncodpers'].values

for c in YTrain_Prod.columns:
    if c != 'ncodpers':
        print('Product ',c)
        y_train = YTrain_Prod[c]
        x_train = YTrain_Prod.drop([c, 'ncodpers'],axis=1)
        
        clf = LogisticRegression()
        clf.fit(x_train, y_train)

        #Stores Probability Value, [:,1]-> to not to gate index values
        p_train = clf.predict_proba(x_train)[:,1]
        
        models[c] = clf
        #Storing Probabilities of Each Model For each User 
        model_preds[c] = p_train

        for id, p in zip(ids, p_train):
            id_preds[id].append(p)
            
        print(roc_auc_score(y_train, p_train))


Product  ind_ahor_fin_ult1
0.9805303597050533
Product  ind_aval_fin_ult1
0.6280712797602849
Product  ind_cco_fin_ult1
0.8198924304471955
Product  ind_cder_fin_ult1


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9737476675076474
Product  ind_cno_fin_ult1
0.8691008476243323
Product  ind_ctju_fin_ult1


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7038644498354911
Product  ind_ctma_fin_ult1


...

0.776659255126004
Product  ind_viv_fin_ult1


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9979707712180714
Product  ind_nomina_ult1


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9987060042330996
Product  ind_nom_pens_ult1
1.0
Product  ind_recibo_ult1
1.0


**Model 2**

In [7]:
#Store List of Models
models2 = {}
#Stores List of Probabilities of each model
model_preds2 = {}
#creating default dictionary to store ids of predictions
id_preds2 = defaultdict(list)
#Stores unique ids

ids2 = XTrain_Cust['ncodpers'].values

for c in YTrain_Prod.columns:
    if c != 'ncodpers':
        print('Product ',c)
        y_train2 = YTrain_Prod[c]
        x_train2 = XTrain_Cust.drop('ncodpers',axis=1)
        
        clf2 = LogisticRegression()
        clf2.fit(x_train2, y_train2)

        #Stores Probability Value, [:,1]-> to not to gate index values
        p_train2 = clf2.predict_proba(x_train2)[:,1]
        
        models2[c] = clf2
        #Storing Probabilities of Each Model For each User 
        model_preds2[c] = p_train2

        for id, p in zip(ids2, p_train2):
            id_preds2[id].append(p)
            
        print(roc_auc_score(y_train2, p_train2))


Product  ind_ahor_fin_ult1


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.9871907318699622
Product  ind_aval_fin_ult1


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.8064280700163626
Product  ind_cco_fin_ult1


...

0.8831927375847852
Product  ind_nom_pens_ult1


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.8947669038338779
Product  ind_recibo_ult1


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.8947669038338779


**From Given Discription of Data:**
- Big Problem in the Santander Product Recommendation was, Some Customers were getting lots of recommendations but some were hardly getting any. 
- To resolve this we are creating a threshold of 7 , So we'll be recommending 7 Products to each User
- In this new Solution, We'll be recommending the products which are not bought by the user. We'll name it as nonActive Products


In [14]:
#Creating Dictionary of already bought Products
already_active = {}
for row in YTrain_Prod.values:
    row = list(row)
    id = row.pop(0)
    active = []
    for c in zip(YTrain_Prod.columns[1:], row):
      if (c[1] > 0):
        active.append(c[0])
    already_active[id] = active

#Taking Average of Probabiities
id_Pred_Final = {}
for i,j in id_preds.items():
  id_Pred_Final[i] = list((np.array(id_preds[i])+np.array(id_preds2[i]))/2)

In [19]:
train_preds = {}

#To sort Dictionaries-> https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
for id, p in id_Pred_Final.items():
  NonActiveProds = {}
  NonActiveProds_sorted = {}
  for i in zip(YTrain_Prod.columns[1:], p):
    if i[0] not in already_active[id]:
      NonActiveProds[i[0]]=i[1]
  
  #Getting Top 7 Elements
  NonActiveProds_sorted = {k: v for k, v in sorted(NonActiveProds.items(), key=lambda item: item[1],reverse=True)[:7]}
  preds = list(NonActiveProds_sorted.keys())
  train_preds[id] = preds

In [120]:
test_preds = {}
YPred_Dict = {}
for row in YTrain_Prod.values:
    id = row[0]
    if id in train_preds.keys():
      p = train_preds[id]
    else:
      #For New ID Recommending Most bought Products, Find More Deails in EDA
      p = ['ind_aval_fin_ult1','ind_ctma_fin_ult1','ind_cder_fin_ult1','ind_dela_fin_ult1','ind_recibo_ult1','ind_nom_pens_ult1','ind_deme_fin_ult1']
    YPred_Dict[id] = p
    test_preds[id] = ' '.join(p)

In [21]:
test_preds = {int(k):v for k,v in test_preds.items()}

**Saving the Predictions file as Text File**

In [124]:
'''from tqdm import tqdm
import json
json_object = json.dumps(test_preds, indent = 4)

try:
    geeky_file = open('Predictions.txt', 'wt')
    geeky_file.write(geeky_file)
    geeky_file.close()
  
except:
    print("Unable to write to file")'''

In [27]:
'''from google.colab import files
files.download("Predictions.txt")'''